In [1]:
import numpy as np
import ananke as an
an.__version__

'0.1.0.dev1'

We define here some dummy input data. Py-ananke has a method to produce such data.

In [2]:
np.random.seed(0)
p = an.Ananke.make_dummy_particles_input()
p.keys()

dict_keys(['pos3', 'vel3', 'mass', 'age', 'feh', 'helium', 'carbon', 'nitrogen', 'oxygen', 'neon', 'magnesium', 'silicon', 'sulphur', 'calcium', 'alpha', 'parentid', 'dform', 'log10_NH'])

The input data must be formatted as a dictionary of equal-length arrays. The dictionary must have the following entries:
- key `pos3`: particle position coordinates in $kpc$ (shape Nx3)
- key `vel3`: particle velocity coordinates in $km.s^{-1}$ (shape Nx3)
- key `mass`: particle stellar mass in solar masses
- key `age`: particle log10 stellar age in years
- key `feh`: particle stellar metallicity \[Fe/H\] in dex relative to solar

Additionally, the following entries can optionally be added:
- key `parentid`: index to give to the parent particle
- key `id`: additional index to classify the parent particle
- key `log10_NH`: log10 hydrogen column densities between Observer position and particle in $cm^{-2}$ - must be provided to estimate extinctions
- key `dform`: particle formation distance
- keys `helium`, `carbon`, `nitrogen`, `oxygen`, `neon`, `magnesium`, `silicon`, `sulphur`, `calcium`: particle various chemical abundances \[X/H\]
- key `alpha`: particle alpha chemical abundances \[Mg/Fe\]

We can define here some parameters for Ananke such as
 - the observer position `observer`
 - the shell of particles to mask `rshell`
 - the sampling factor `fsample`
 - the photometric system of choise `photo_sys`
 - the CMD `cmd_magnames` and its box limits `cmd_box`

In [3]:
D = 200 # *units.kpc

observer = np.nan*np.ones(3)
while not np.linalg.norm(observer)<1:
    observer = 2*np.random.rand(3)-1

observer *= D/np.linalg.norm(observer)

rshell = [0, 2*D]

fsample = 0.01

photo_sys = 'padova/GAIADR2'

cmd_magnames = {'magnitude': 'Gmag',
                'color_minuend': 'G_BPmag',
                'color_subtrahend': 'G_RPmag'}

cmd_box = {
           'abs_mag_lim_lo': -1000,
           'abs_mag_lim_hi': 1000,
        #    'app_mag_lim_lo' : -1000,
           'app_mag_lim_hi': 30,
        #    'color_lim_lo' : -1000,
        #    'color_lim_hi' : 1000
           }

For more details regarding these parameters and more, you may consult the docstring associated to the class `Ananke`:

In [4]:
help(an.Ananke)

Help on class Ananke in module ananke.Ananke:

class Ananke(builtins.object)
 |  Ananke(particles, name, ngb=64, d_params={}, e_params={}, err_params={}, **kwargs) -> None
 |  
 |  Represents a single ananke pipeline.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, particles, name, ngb=64, d_params={}, e_params={}, err_params={}, **kwargs) -> None
 |      Parameters
 |      ----------
 |      particles : dict
 |          A dictionary of same-length arrays representing particles
 |          data of a stellar population - see notes for formatting
 |      
 |      name : str
 |          Name for the pipeline
 |      
 |      ngb : int
 |          Number of neighbours to use in kernel density estimation
 |      
 |      d_params : dict
 |          Parameters to configure the kernel density estimation. Use
 |          class method display_density_docs to find what parameters can
 |          be defined
 |      
 |      e_params : dict
 |          Parameters to configure the extinction

Preparing now the pipeline with GAIA photometry

In [5]:
name = 'sim'
ananke = an.Ananke(p, name, fsample=fsample,
                   observer=observer, rshell=rshell,
                   photo_sys=photo_sys, cmd_magnames=cmd_magnames,
                   **cmd_box)

/home/athob/miniconda3/envs/main/lib/python3.10/site-packages/ananke/utils.py:21: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._record_of_all_used_keys = set()
/home/athob/miniconda3/envs/main/lib/python3.10/site-packages/ananke/utils.py:21: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._record_of_all_used_keys = set()


The method `run` runs the pipeline:

In [6]:
survey = ananke.run()

Dimensions = 3
Normalization constant of Kernel type 3: 0.596831
Reading ASCII format file: to_enbid 
Read 300000 records
Allocated 3.8147 MByte for particle storage.
Type = 1 Particles = 100000
Total particles = 100000
Allocated 22.1252 MByte for Binary-Tree and Entropy bins.

Scaling Co-ordinates as x[i]=x[i]/h[i] with h[i]->
1 1 1 

Starting to Build Tree .......
Particle Type = 1 First node number = 1
                  Last node number  = 199999
Total number of nodes = 199999
Treebuild time = 0.134453 s 

Density Calculation. Smoothing .....
Evaluated =   1 % Time Left = 0.617853 s of 0.624100 s Par no = 96805 Density = 4.540412e-02 
Evaluated =   2 % Time Left = 0.608819 s of 0.621250 s Par no = 55454 Density = 4.964532e-03 
Evaluated =   3 % Time Left = 0.663085 s of 0.683600 s Par no = 27717 Density = 1.827217e-01 
Evaluated =   4 % Time Left = 0.678305 s of 0.706575 s Par no = 42234 Density = 2.712376e-02 
Evaluated =   5 % Time Left = 0.661649 s of 0.696480 s Par no = 99785 De

nbody1/sim/

	1	1

path nbody1/sim/ sats 1 x[0] 1


Isochrone Grid Size:                (Age bins=71,Feh bins=34,Alpha bins=1)
Time Isochrone Reading              2.18597     
------------------------------
nbody1/sim/sim.ebf  Sat No=0
Particles=100000
Satellite Info
Particles=100000 Mass=5.49989e+08 0.379706
Total Stars=2421317 accepted=1614299 rejected=807018
-----------Done---------------
Total stars written                 1614299                 
30
File written-                       /home/athob/Software/repos/py-ananke/jupyter/survey.sim.ebf
Calculating magnitudes in GAIADR2 system................
initializing isochrone data
interpolating on isochrone tables
Appending spherical coordinates................
Total Time=                         5.82654     
Exported the following quantities to /home/athob/Software/repos/py-ananke/jupyter/survey.sim.h5
['age', 'alpha', 'calcium', 'carbon', 'dec', 'dform', 'dmod', 'feh', 'gaiadr2_g_bpmag', 'gaiadr2_g_rpmag', 'gaiadr2_gmag', 'glat', 'glon', 'grav', 'helium', 'lum', 'mact', 'magnesium', 

/home/athob/miniconda3/envs/main/lib/python3.10/site-packages/ananke/_default_error_model.py:37: RuntimeWarning: overflow encountered in square
  grvs = rpmag + 132.32 - 377.28*ggrp + 402.32*ggrp**2 - 190.97*ggrp**3 + 34.026*ggrp**4
/home/athob/miniconda3/envs/main/lib/python3.10/site-packages/ananke/_default_error_model.py:37: RuntimeWarning: overflow encountered in multiply
  grvs = rpmag + 132.32 - 377.28*ggrp + 402.32*ggrp**2 - 190.97*ggrp**3 + 34.026*ggrp**4
/home/athob/miniconda3/envs/main/lib/python3.10/site-packages/ananke/_default_error_model.py:37: RuntimeWarning: overflow encountered in power
  grvs = rpmag + 132.32 - 377.28*ggrp + 402.32*ggrp**2 - 190.97*ggrp**3 + 34.026*ggrp**4
/home/athob/miniconda3/envs/main/lib/python3.10/site-packages/ananke/_default_error_model.py:37: RuntimeWarning: invalid value encountered in subtract
  grvs = rpmag + 132.32 - 377.28*ggrp + 402.32*ggrp**2 - 190.97*ggrp**3 + 34.026*ggrp**4
/home/athob/miniconda3/envs/main/lib/python3.10/site-package

Exported the following quantities to /home/athob/Software/repos/py-ananke/jupyter/survey.sim.h5
['dec_Sig', 'pi_Err', 'gaiadr2_g_bpmag_Sig', 'mura_Err', 'pi_Sig', 'gaiadr2_g_rpmag_Sig', 'ra_Sig', 'mudec_Err', 'ra_Err', 'vr_Err', 'gaiadr2_g_bpmag_Err', 'mudec_Sig', 'mura_Sig', 'vr_Sig', 'gaiadr2_gmag_Err', 'gaiadr2_g_rpmag_Err', 'dec_Err', 'gaiadr2_gmag_Sig']
Overwritten the following quantities to /home/athob/Software/repos/py-ananke/jupyter/survey.sim.h5
('gaiadr2_gmag', 'gaiadr2_g_rpmag', 'gaiadr2_g_bpmag', 'pi', 'ra', 'dec', 'mura', 'mudec', 'vr')
Overwritten the following quantities to /home/athob/Software/repos/py-ananke/jupyter/survey.sim.h5
('glon', 'glat', 'mul', 'mub')


The output is saved as a `vaex` dataframe, with its columns organized in alphabetical order. These notably include:
- key `A_0` for the reference extinction which extinction coefficients are based on (at $\lambda = 550 \, nm$ in the case of Gaia DR2)
- key `A_{filter_name}` for the extinction value in each filter designated by `filter_name` (in this case, where `filter_name` is one of the 3 Gaia bands `gaia_g_bpmag`, `gaia_g_rpmag` \& `gaia_gmag`)
- key `E(B-V)` for the reddening index
- key `age` for the log10 stellar age in years
- key `alpha`, `calcium`, `carbon`, `helium`, `magnesium`, `neon`, `nitrogen`, `oxygen`, `silicon`, `sulphur` for the various chemical abundances as given as input
- key `dec`, `ra` for the astrometric declination and right ascension celestial coordinates in degrees
- key `dform` for the formation distance as given as input
- key `dmod` for the distance modulus
- key `feh` for the stellar metallicity \[Fe/H\] in dex relative to solar
- key `glat`, `glon` for the astrometric galactic latitude and longitude celestrial coordinates in degrees
- key `grav` for the log10 surface gravity in CGS units
- key `log10_NH` for the log10 hydrogen column density between Observer position and star in $cm^{-2}$
- key `lum` for the stellar luminosity in solar luminosities
- key `mact`, `mtip`, `mini` for respectively the current stellar mass, the mass of that same star at tip of giant branch for its given age \& metallicity and its stellar mass on zero-age main sequence, all in solar masses
- key `mub`, `mudec`, `mul`, `mura` for the astrometric proper motions, respectively in the direction of the galactic latitude, declination, galactic longitude and right ascension, all in milliarcseconds per year
- key `parentid` for the parent particle index as given as input
- key `partid` for the flag that identifies stars that are *not* central relatively to their parent particle
- key `pi` for the star parallax in milliarcseconds
- key `px`, `py`, `pz` for the star position cartesian coordinates in $kpc$ relative to the Observer's position
- key `rad` for the star distance to the Observer in $kpc$
- key `teff` for the star effective temperature in Kelvin
- key `vr` for the star astrometric radial velocity in $km.s^{-1}$
- key `vx`, `vy`, `vz` for the star velocity cartesian coordinates in $km.s^{-1}$ relative to the Observer's velocity

Additionally, astrometric and photometric quantities `X` all have associated columns identified as:
- key `X_Sig` for the standard error on the quantity `X`
- key `X_Err` for the actual drawn gaussian error on the quantity `X`

In [7]:
survey

#,A_0,A_gaiadr2_g_bpmag,A_gaiadr2_g_rpmag,A_gaiadr2_gmag,E(B-V),age,alpha,calcium,carbon,dec,dec_Err,dec_Sig,dform,dmod,feh,gaiadr2_g_bpmag,gaiadr2_g_bpmag_Err,gaiadr2_g_bpmag_Intrinsic,gaiadr2_g_bpmag_Sig,gaiadr2_g_rpmag,gaiadr2_g_rpmag_Err,gaiadr2_g_rpmag_Intrinsic,gaiadr2_g_rpmag_Sig,gaiadr2_gmag,gaiadr2_gmag_Err,gaiadr2_gmag_Intrinsic,gaiadr2_gmag_Sig,glat,glon,grav,helium,log10_NH,lum,mact,magnesium,mtip,mub,mudec,mudec_Err,mudec_Sig,mul,mura,mura_Err,mura_Sig,neon,nitrogen,oxygen,parentid,partid,pi,pi_Err,pi_Sig,px,py,pz,ra,ra_Err,ra_Sig,rad,silicon,smass,sulphur,teff,vr,vr_Err,vr_Sig,vx,vy,vz
0,0.7093970129666665,0.704672944261162,0.4033522358845791,0.5137419405527027,0.2288377461182795,10.016325950622559,0.3380742371082306,-1.1830222606658936,-0.39430123567581177,nan,nan,nan,0.0,20.87123488088669,-0.8867433071136475,nan,nan,8.817544,nan,nan,nan,7.321502,nan,nan,nan,8.115389,nan,nan,nan,62144.36,-0.7315406799316406,21.757467670308866,0.056668214,0.4809227,-0.5486690402030945,0.886202,nan,nan,nan,nan,nan,nan,nan,nan,-0.7326117157936096,-0.652482807636261,-0.6697278618812561,0,0,nan,nan,nan,-142.03525597960677,46.20841677042607,0.692591828190988,nan,nan,nan,149.36435788052748,-0.7563512325286865,0.4812842042141955,-1.0924330949783325,3.6188776,nan,nan,nan,80.28685760498047,-54.7354850769043,62.91861343383789
1,0.184335809131283,0.18396131598009025,0.10509330221656336,0.1347890384517113,0.059463164235897745,10.016325950622559,0.3380742371082306,-1.1830222606658936,-0.39430123567581177,nan,nan,nan,0.0,20.90373906133466,-0.8867433071136475,nan,nan,9.0512905,nan,nan,nan,7.498887,nan,nan,nan,8.315174,nan,nan,nan,64744.305,-0.7315406799316406,21.17218802443275,0.048167586,0.4630728,-0.5486690402030945,0.886202,nan,nan,nan,nan,nan,nan,nan,nan,-0.7326117157936096,-0.652482807636261,-0.6697278618812561,0,1,nan,nan,nan,-143.77167469460034,47.82129448048966,-5.525834122396032,nan,nan,nan,151.6169696736745,-0.7563512325286865,0.4635413655187806,-1.0924330949783325,3.6097207,nan,nan,nan,83.06969313228834,-39.06569347107564,44.08486449645322
2,0.6506825002228606,0.6918469008156594,0.3844989040361987,0.5412446602487346,0.20989758071705178,10.016325950622559,0.3380742371082306,-1.1830222606658936,-0.39430123567581177,nan,nan,nan,0.0,20.851514561194136,-0.8867433071136475,nan,nan,5.507653,nan,nan,nan,4.7665434,nan,nan,nan,5.2235985,nan,nan,nan,34601.574,-0.7315406799316406,21.719947441597984,0.62339365,0.73794055,-0.5486690402030945,0.886202,nan,nan,nan,nan,nan,nan,nan,nan,-0.7326117157936096,-0.652482807636261,-0.6697278618812561,0,1,nan,nan,nan,-140.8445734092126,45.49169054835579,1.2117012727306524,nan,nan,nan,148.01403983320222,-0.7563512325286865,0.7378524315679198,-1.0924330949783325,3.7689726,nan,nan,nan,80.27981120824414,-56.78730083022977,43.2034337518152
3,0.5853051731255048,0.57426916654755,0.3275537885553172,0.40681096522497545,0.18880812036306605,10.016325950622559,0.3380742371082306,-1.1830222606658936,-0.39430123567581177,nan,nan,nan,0.0,20.819685704214336,-0.8867433071136475,nan,nan,9.908007,nan,nan,nan,8.157672,nan,nan,nan,9.043117,nan,nan,nan,77813.87,-0.7315406799316406,21.67396067741074,0.027216235,0.39234725,-0.5486690402030945,0.886202,nan,nan,nan,nan,nan,nan,nan,nan,-0.7326117157936096,-0.652482807636261,-0.6697278618812561,0,1,nan,nan,nan,-141.5308932930025,35.10464710311902,-3.4497639495419588,nan,nan,nan,145.86031288824708,-0.7563512325286865,0.3928981627574019,-1.0924330949783325,3.585806,nan,nan,nan,82.79570902099319,-45.853085149179805,70.23675579370128
4,0.38734713403208176,0.4185867733363559,0.23065183690015303,0.3326465298503053,0.12495068839744572,10.016325950622559,0.3380742371082306,-1.1830222606658936,-0.39430123567581177,nan,nan,nan,0.0,20.801481352393886,-0.8867433071136475,nan,nan,4.388494,nan,nan,nan,3.7630243,nan,nan,nan,4.157759,nan,nan,nan,18834.406,-0.7315406799316406,21.49467866182269,1.6688203,0.8256626,-0.5486690402030945,0.886202,nan,nan,nan,nan,nan,nan,nan,nan,-0.7326117157936096,-0.6524828

Please refer to [`vaex`'s documentation](https://vaex.io/docs/tutorial.html) for further help on how to use `vaex` dataframes: the following line for example isolate only the rows with non-NaN photometry.

In [8]:
survey[~survey.gaiadr2_gmag.isna()]

#,A_0,A_gaiadr2_g_bpmag,A_gaiadr2_g_rpmag,A_gaiadr2_gmag,E(B-V),age,alpha,calcium,carbon,dec,dec_Err,dec_Sig,dform,dmod,feh,gaiadr2_g_bpmag,gaiadr2_g_bpmag_Err,gaiadr2_g_bpmag_Intrinsic,gaiadr2_g_bpmag_Sig,gaiadr2_g_rpmag,gaiadr2_g_rpmag_Err,gaiadr2_g_rpmag_Intrinsic,gaiadr2_g_rpmag_Sig,gaiadr2_gmag,gaiadr2_gmag_Err,gaiadr2_gmag_Intrinsic,gaiadr2_gmag_Sig,glat,glon,grav,helium,log10_NH,lum,mact,magnesium,mtip,mub,mudec,mudec_Err,mudec_Sig,mul,mura,mura_Err,mura_Sig,neon,nitrogen,oxygen,parentid,partid,pi,pi_Err,pi_Sig,px,py,pz,ra,ra_Err,ra_Sig,rad,silicon,smass,sulphur,teff,vr,vr_Err,vr_Sig,vx,vy,vz
0,0.7201175332318196,0.7134542903541995,0.4084361423413181,0.5178562640788976,0.23229597846187727,9.095987319946289,-0.017641598358750343,-0.8040515184402466,-0.8312878608703613,54.719234,1.1448103463624403e-07,8.156472e-08,0.0,21.24488070012207,-0.6019725203514099,20.17186,0.14475732663937507,-1.9312339,0.09907752,18.310198,0.14402777384218768,-3.464119,0.09907752,19.104248,-0.0003271873134447138,-2.6581607,0.006084603,-0.11579015,146.25659,10.500975,0.018333230167627335,21.763981699974927,1283.2991,1.901,-0.6196141242980957,1.7589691,-0.11884970757054293,-0.28449323403569793,-0.19208180636421257,0.5654477,0.31195892947495524,0.17466317425003625,0.20043543039102113,0.5654477,-0.4461532533168793,-0.9858365058898926,-0.5438077449798584,28,1,0.11883797772145974,0.1132012914942843,0.29363298,-147.52139845290782,98.5460961576138,-0.3585239696845868,56.1353,4.140486431147566e-08,8.156472e-08,177.4092010264521,-0.1619892418384552,1.9010364494309275,-0.6848231554031372,3.615165,nan,nan,nan,-60.96085862558914,4.353080367301258,-74.9092985289255
1,0.33403628400303675,0.33346713582144344,0.19060551078026267,0.24470128261930946,0.1077536400009796,9.548486709594727,-0.14476357400417328,-0.4364003837108612,-0.7601265907287598,17.434006,9.928033961790062e-08,2.5173674e-07,0.0,21.739221738307855,-0.4794479310512543,21.29973,0.25238436516191837,-1.0253428,0.24261825,19.841814,0.45283295374079735,-2.536398,0.24261825,20.250189,0.00495977394107601,-1.7386937,0.011473365,-22.126055,178.0618,17.235332,-0.8525111675262451,21.430371958554815,544.5688,1.3085884,-0.6242115497589111,1.3095868,0.8915441865930587,-1.1783465169814455,-1.198450963925625,1.505447,2.2893589905723593,2.1558095854965535,2.11482805730037,1.505447,-0.029001595452427864,-0.3071211576461792,-1.4820371866226196,118,1,-0.16816188360792872,-0.17265094611134355,0.9062522,-206.24072097285307,6.979381176550179,-83.9029328423293,65.66848,1.5394333791304828e-07,2.5173674e-07,222.7636615082322,-0.6725741624832153,1.3160010522070515,-0.19469265639781952,3.6164653,nan,nan,nan,-53.42110813744579,-9.803716623295626,28.932186341365828
2,0.6271200139981715,0.6465330999897427,0.36553625835222914,0.49270646119023725,0.20229677870908758,8.785699844360352,0.8913701772689819,-0.08881347626447678,-1.046112298965454,11.806886,-5.461239693336942e-08,4.181088e-07,0.0,21.54159360667728,-0.8756767511367798,21.382792,0.04448076293525185,-0.8498174,0.35615402,20.513166,0.5324325641105968,-1.9126871,0.35615402,20.735762,0.004213228189661257,-1.3027523,0.015076785,-17.069452,-170.51006,123.72461,-0.2251279205083847,21.70392897597074,272.31705,2.292733,0.01569346897304058,2.2908814,-1.4806178225379107,1.3564791474190814,1.3553592603149776,2.3408535,-2.6049850053772996,-2.6717298594083205,-2.586740042055322,2.3408535,-1.1234924793243408,-1.248551368713379,-1.0234960317611694,127,1,0.11209902957558333,0.10718224388855435,1.5051917,-191.76492242701607,-32.0558971556688,-59.699701881255734,76.4974,-1.3447410821521523e-07,4.181088e-07,203.38490706196743,-1.1956217288970947,2.2934981458798505,-0.9377636909484863,3.6943395,nan,nan,nan,42.06984570359715,53.763096237093634,-55.41853942761457
3,0.7347135856072053,0.7250212324297668,0.41498626743537254,0.5223546911188022,0.2370043824539372,9.391923904418945,0.3202667236328125,-0.5372077822685242,-0.34959182143211365,20.089922,8.27768965429226e-07,3.1524527e-07,0.0,21.77074680